<a href="https://colab.research.google.com/github/malraharsh/Pruning-Experiments/blob/master/Large_Model_Trainig_OF_Pruning_Experiments_on_DogsvsCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q tensorflow-model-optimization

import tempfile
import os

import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
import pandas as pd


from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from tensorflow import keras
import datetime

%load_ext tensorboard

import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude, ConstantSparsity

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

import os
import numpy as np
import matplotlib.pyplot as plt



from IPython.display import display

SHOW = False

     |████████████████████████████████| 174kB 3.3MB/s 
     |████████████████████████████████| 296kB 10.5MB/s 


In [2]:
def makedir(path):
    os.makedirs(path, exist_ok=True)

In [3]:
# path_saved_model = '/content/drive/My Drive/Colab Notebooks/Pruning Experiments/training_cp-1000train, 2000val, 300epochs---0250.h5'

# makedir('saved_models')
# !cp '/content/drive/My Drive/Colab Notebooks/Pruning Experiments/training_cp-1000train, 2000val, 300epochs---0250.h5' 'saved_models'
# drive.flush_and_unmount()
# path_saved_model = '/content/saved_models/training_cp-1000train, 2000val, 300epochs---0250.h5'

In [4]:
log_train = 'logs/train/'
log_prune = 'logs/prune/'

makedir(log_train)
makedir(log_prune)

In [5]:
#https://www.tensorflow.org/tutorials/images/classification

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

68608000/68606236 [==============================] - 1s 0us/step


In [6]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
# train_dir = os.path.join(PATH, 'validation')
# validation_dir = os.path.join(PATH, 'train')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [7]:
batch_size = 64
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

# train_image_generator = ImageDataGenerator(
#                     rescale=1./255,
#                     rotation_range=45,
#                     width_shift_range=.15,
#                     height_shift_range=.15,
#                     horizontal_flip=True,
#                     zoom_range=0.5)

validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data


# train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
#                                                            directory=train_dir,
#                                                            shuffle=True,
#                                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
#                                                            class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')



# def change_pct_test():
#     train_image_generator = ImageDataGenerator(validation_split=pct_test, 
#                         rescale=1./255,
#                         rotation_range=45,
#                         width_shift_range=.15,
#                         height_shift_range=.15,
#                         horizontal_flip=True,
#                         zoom_range=0.5)


#     train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
#                                                             directory=train_dir,
#                                                             shuffle=True,
#                                                             target_size=(IMG_HEIGHT, IMG_WIDTH),
#                                                             class_mode='binary',
#                                                             subset='training')

#     val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
#                                                                 directory=validation_dir,
#                                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
#                                                                 class_mode='binary',
#                                                                 subset='validation')


Found 1000 images belonging to 2 classes.


In [8]:
def get_train_model():

    model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model

def get_cifar_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model


def get_cifar_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(layers.Dense(1))

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    # model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes

    # model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [9]:
def train(model=None):#train_images, train_labels, test_images, test_labels):

    train_image_generator = ImageDataGenerator(validation_split=pct_test, 
                        rescale=1./255,
                        rotation_range=45,
                        width_shift_range=.15,
                        height_shift_range=.15,
                        horizontal_flip=True,
                        zoom_range=0.5)


    train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                            directory=train_dir,
                                                            shuffle=True,
                                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                            class_mode='binary',
                                                            subset='training')

    val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                                directory=train_dir,
                                                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                                class_mode='binary',
                                                                subset='validation')


    if model is None:
        model = get_cifar_model() # get_train_model()

    # log_dir = log_train + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_train, histogram_freq=1)

    history = model.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // batch_size,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // batch_size)
    # callbacks=[tensorboard_callback])
    
    return model, history.history    

In [10]:
def prune(model=None):


    train_image_generator = ImageDataGenerator(validation_split=pct_test, 
                        rescale=1./255,
                        rotation_range=45,
                        width_shift_range=.15,
                        height_shift_range=.15,
                        horizontal_flip=True,
                        zoom_range=0.5)


    train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                            directory=train_dir,
                                                            shuffle=True,
                                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                            class_mode='binary',
                                                            subset='training')

    val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                                directory=train_dir,
                                                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                                class_mode='binary',
                                                                subset='validation')


    if model is None:
        model = get_train_model()


    # Define model for pruning.
    pruning_params = {
        'pruning_schedule': ConstantSparsity(pct_prune, 0), #target, begin step
        'block_size': (1, 1),
        'block_pooling_type': 'AVG'}

    end_step = np.ceil(total_val / batch_size).astype(np.int32) * epochs

    pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
                                                               final_sparsity=pct_prune,
                                                               begin_step=0,
                                                               end_step=end_step)}


    model_for_pruning = prune_low_magnitude(model, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
      tfmot.sparsity.keras.PruningSummaries(log_dir=log_prune),
    ]

    history = model_for_pruning.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // batch_size,
    epochs=EPOCHS_PRUNE,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // batch_size,
    callbacks=callbacks,
    verbose=VERBOSE)

    return model_for_pruning, history.history

In [11]:
EPOCHS = 1
EPOCHS_PRUNE = 2
VERBOSE = 1
SHOW = 0
pct_prune = 0.05
pct_test = 0.2

In [12]:
total_train, total_val

(2000, 1000)

In [ ]:
# EPOCHS = 30
# pct_test = 0.3
# model, dt = train()

In [ ]:
EPOCHS = 30
pct_test = 0.2
model, dt = train(model)

In [15]:
EPOCHS = 30
pct_test = 0.2
model, dt = train(model)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/30
25/25 [==============================] - 14s 567ms/step - loss: 0.4041 - accuracy: 0.8019 - val_loss: 0.5517 - val_accuracy: 0.7188
Epoch 2/30
25/25 [==============================] - 14s 565ms/step - loss: 0.4091 - accuracy: 0.8056 - val_loss: 0.5659 - val_accuracy: 0.7005
Epoch 3/30
25/25 [==============================] - 14s 565ms/step - loss: 0.3964 - accuracy: 0.8100 - val_loss: 0.6359 - val_accuracy: 0.7214
Epoch 4/30
25/25 [==============================] - 14s 568ms/step - loss: 0.4004 - accuracy: 0.8150 - val_loss: 0.6299 - val_accuracy: 0.6901
Epoch 5/30
25/25 [==============================] - 14s 562ms/step - loss: 0.4062 - accuracy: 0.8069 - val_loss: 0.7532 - val_accuracy: 0.6250
Epoch 6/30
25/25 [==============================] - 14s 561ms/step - loss: 0.4083 - accuracy: 0.8075 - val_loss: 0.6632 - val_accuracy: 0.6589
Epoch 7/30
25/25 [==============================] - 14s 565

In [ ]:
'''
loss: 0.5314 - accuracy: 0.7147 - val_loss: 0.9066 - val_accuracy: 0.5260
loss: 0.5672 - accuracy: 0.7024 - val_loss: 1.5961 - val_accuracy: 0.5000
'''

In [14]:
# path_cp = f"training_cp-{EPOCHS:04d}.h5"
# model.save(path_cp)

In [ ]:
# model = tf.keras.models.load_model(path_saved_model)

In [ ]:
# model.evaluate(train_data_gen, verbose=1)

In [21]:
path_cp = f"training_cp-0.2-90.h5"
model.save(path_cp)

In [23]:
!cp '/content/training_cp-0.2-90.h5' '/content/drive/My Drive/Colab Notebooks/Pruning Experiments/training_cp-0.2-90.h5'

In [16]:

# !cp '/content/drive/My Drive/Colab Notebooks/Pruning Experiments/training_cp-0.2-90.h5' /content/
# drive.flush_and_unmount()
model = tf.keras.models.load_model('/content/training_cp-0.2-90.h5')

In [17]:
model.evaluate(val_data_gen, verbose=1)

16/16 [==============================] - 3s 182ms/step - loss: 0.5361 - accuracy: 0.7800


[0.5360532999038696, 0.7799999713897705]

In [18]:
EPOCHS_PRUNE = 3
pct_prune = 0.05
mp, dp = prune(model)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Instructions for updating:
Please use `layer.add_weight` method instead.
Epoch 1/3
25/25 [==============================] - 17s 673ms/step - loss: 0.3223 - accuracy: 0.8606 - val_loss: 0.5376 - val_accuracy: 0.7630
Epoch 2/3
25/25 [==============================] - 17s 670ms/step - loss: 0.3145 - accuracy: 0.8594 - val_loss: 0.5370 - val_accuracy: 0.7786
Epoch 3/3
25/25 [==============================] - 17s 668ms/step - loss: 0.2875 - accuracy: 0.8763 - val_loss: 0.5798 - val_accuracy: 0.7682


In [19]:
pct_prune = 0.1
mp, dp = prune(model)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/3
25/25 [==============================] - 17s 687ms/step - loss: 0.3246 - accuracy: 0.8487 - val_loss: 0.4928 - val_accuracy: 0.7969
Epoch 2/3
25/25 [==============================] - 17s 672ms/step - loss: 0.3019 - accuracy: 0.8650 - val_loss: 0.7637 - val_accuracy: 0.7135
Epoch 3/3
25/25 [==============================] - 17s 666ms/step - loss: 0.2822 - accuracy: 0.8712 - val_loss: 0.5877 - val_accuracy: 0.7760


In [20]:
pct_prune = 0.2
mp, dp = prune(model)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/3
25/25 [==============================] - 17s 694ms/step - loss: 0.3099 - accuracy: 0.8519 - val_loss: 0.5184 - val_accuracy: 0.7552
Epoch 2/3
25/25 [==============================] - 17s 675ms/step - loss: 0.2902 - accuracy: 0.8694 - val_loss: 0.4141 - val_accuracy: 0.8047
Epoch 3/3
25/25 [==============================] - 17s 675ms/step - loss: 0.2773 - accuracy: 0.8744 - val_loss: 0.5754 - val_accuracy: 0.7630


In [21]:
EPOCHS_PRUNE = 10
pct_prune = 0.1
mp, dp = prune(model)

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/10
25/25 [==============================] - 18s 701ms/step - loss: 0.3043 - accuracy: 0.8644 - val_loss: 0.5624 - val_accuracy: 0.7292
Epoch 2/10
25/25 [==============================] - 17s 682ms/step - loss: 0.2783 - accuracy: 0.8750 - val_loss: 0.7054 - val_accuracy: 0.7396
Epoch 3/10
25/25 [==============================] - 17s 676ms/step - loss: 0.2727 - accuracy: 0.8763 - val_loss: 0.6001 - val_accuracy: 0.7552
Epoch 4/10
25/25 [==============================] - 17s 680ms/step - loss: 0.2694 - accuracy: 0.8813 - val_loss: 0.5466 - val_accuracy: 0.7604
Epoch 5/10
25/25 [==============================] - 17s 676ms/step - loss: 0.2821 - accuracy: 0.8656 - val_loss: 0.9704 - val_accuracy: 0.6276
Epoch 6/10
25/25 [==============================] - 17s 679ms/step - loss: 0.2819 - accuracy: 0.8731 - val_loss: 0.5981 - val_accuracy: 0.7292
Epoch 7/10
25/25 [==============================] - 17s 678

In [ ]:
# %tensorboard --logdir /content/logs/train

In [ ]:
# %tensorboard --logdir /content/logs/prune